In [84]:
import os
import pandas as pd
from tqdm import tqdm

In [85]:
from GurobiSolver import *

In [86]:
parent_dir = os.path.dirname(os.getcwd())
os.makedirs(parent_dir + '/Results', exist_ok=True)

# Train data sets

## Networks data

In [87]:
# Load the networks details
tr_net_df = pd.read_csv("Results/TrainNetworksDetails.csv")
tr_net_df.head()

,id,k,|N|,|A|,Level 0,Level 1,Level 2,Level 3,Level 4
0,1,1,42,65,1,5,10,25,1
1,1,2,42,65,1,5,10,25,1
2,1,3,42,65,1,5,10,25,1
3,1,4,42,65,1,5,10,25,1
4,1,5,42,65,1,5,10,25,1


## Decision tree model

In [88]:
# # Initialize directories
# data = "Train"
# ml_alg = "dt"
# network_dir = os.path.join(parent_dir, "Networks", data)
# predictions_dir = os.path.join(parent_dir, "Predictions", ml_alg, data)
# models_dir = os.path.join(parent_dir, "Models-ml", ml_alg, data)
# os.makedirs(models_dir, exist_ok=True)

## Random forest model

In [89]:
# # Initialize directories
# data = "Train"
# ml_alg = "rf"
# network_dir = os.path.join(parent_dir, "Networks", data)
# predictions_dir = os.path.join(parent_dir, "Predictions", ml_alg, data)
# models_dir = os.path.join(parent_dir, "Models-ml", ml_alg, data)
# os.makedirs(models_dir, exist_ok=True)

# Test data sets

## Small networks

In [90]:
# Load the small networks details
st_net_df = pd.read_csv("Results/SmallTestNetworksDetails.csv")
st_net_df.head()

,id,k,|N|,|A|,Level 0,Level 1,Level 2,Level 3,Level 4,Level 5
0,1,1,42,66,1,5,10,25,1,-
1,1,2,42,66,1,5,10,25,1,-
2,1,3,42,65,1,5,10,25,1,-
3,1,4,42,65,1,5,10,25,1,-
4,1,5,42,65,1,5,10,25,1,-


In [91]:
# Initialize directories
data = "Test"
opt_models_dir = os.path.join(parent_dir, "Models-gp", data)
dt_models_dir = os.path.join(parent_dir, "Models-ml", "dt", data)
rf_models_dir = os.path.join(parent_dir, "Models-ml", "rf", data)

### Arc interdiction costs are set to 1

In [92]:
tag = "U"

In [93]:
# Interdiction budget
small_uc_B = [i for i in range(1, 9+1)]

In [94]:
# Results for the small networks with unit costs
cols = ["id", "k", f"B_{tag}", f"Opt_{tag}", f"ObjVal_dt_{tag}", f"ObjVal_rf_{tag}"]
vals = []
for id, k in tqdm(zip(st_net_df["id"], st_net_df["k"])):
    name = f"network_{id}_{k}"
    for B in small_uc_B:
        # Set models directory
        # Optimal model
        opt_model_dir = os.path.join(opt_models_dir, f"Model_{B}{tag}")
        opt_obj = load_json(name, opt_model_dir)["ObjVal"]
        # Decision tree model
        dt_model_dir = os.path.join(dt_models_dir, f"Model_{B}{tag}")
        dt_obj = load_json(name, dt_model_dir)["ObjVal"]
        # Random forest model
        rf_model_dir = os.path.join(rf_models_dir, f"Model_{B}{tag}")
        rf_obj = load_json(name, rf_model_dir)["ObjVal"]
        # Append results
        vals.append([id, k, B, opt_obj, dt_obj, rf_obj])
u_small_df = pd.DataFrame(vals, columns=cols)
u_small_df = u_small_df.sort_values(by=["id", f"B_{tag}", "k"])
u_small_df[f"Gap_dt_{tag}"] = (u_small_df[f"ObjVal_dt_{tag}"] - u_small_df[f"Opt_{tag}"]) / u_small_df[f"Opt_{tag}"] * 100
u_small_df[f"Gap_rf_{tag}"] = (u_small_df[f"ObjVal_rf_{tag}"] - u_small_df[f"Opt_{tag}"]) / u_small_df[f"Opt_{tag}"] * 100
u_small_df.round(2).to_csv(f"Results/SmallTestResults_{tag}.csv", index=False)

50it [00:02, 22.23it/s]


### Arc interdiction costs are greater that or equal to 1

In [95]:
tag = ""

In [96]:
# Interdiction budget
small_B = [5, 10, 20, 30, 40, 80, 120, 150, 200]

In [97]:
# Results for the small networks with costs greater than or equal to 1
cols = ["id", "k", f"B_{tag}", f"Opt_{tag}", f"ObjVal_dt_{tag}", f"ObjVal_rf_{tag}"]
vals = []
for id, k in tqdm(zip(st_net_df["id"], st_net_df["k"])):
    name = f"network_{id}_{k}"
    for B in small_B:
        # Set models directory
        # Optimal model
        opt_model_dir = os.path.join(opt_models_dir, f"Model_{B}{tag}")
        opt_obj = load_json(name, opt_model_dir)["ObjVal"]
        # Decision tree model
        dt_model_dir = os.path.join(dt_models_dir, f"Model_{B}{tag}")
        dt_obj = load_json(name, dt_model_dir)["ObjVal"]
        # Random forest model
        rf_model_dir = os.path.join(rf_models_dir, f"Model_{B}{tag}")
        rf_obj = load_json(name, rf_model_dir)["ObjVal"]
        # Append results
        vals.append([id, k, B, opt_obj, dt_obj, rf_obj])
small_df = pd.DataFrame(vals, columns=cols)
small_df = small_df.sort_values(by=["id", f"B_{tag}", "k"])
small_df[f"Gap_dt_{tag}"] = (small_df[f"ObjVal_dt_{tag}"] - small_df[f"Opt_{tag}"]) / small_df[f"Opt_{tag}"] * 100
small_df[f"Gap_rf_{tag}"] = (small_df[f"ObjVal_rf_{tag}"] - small_df[f"Opt_{tag}"]) / small_df[f"Opt_{tag}"] * 100
small_df.round(2).to_csv(f"Results/SmallTestResults_{tag}.csv", index=False)

50it [00:03, 12.79it/s]


## Large networks

In [98]:
# Load the large networks details
# At each level of the network, the number of nodes, flow capacity range, and interdiction cost range are specified in a respective order.
lt_net_df = pd.read_csv("Results/LargeTestNetworksDetails.csv")
lt_net_df.head()

,id,k,|N|,|A|,Level 0,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9,Level 10
0,6,1,29002,39897,"[1/(50, 100)/(100, 101)]","[1000/(1, 75)/(1, 75)]","[1500/(1, 50)/(1, 50)]","[1500/(1, 40)/(1, 40)]","[2000/(1, 40)/(1, 40)]","[2500/(1, 30)/(1, 30)]","[2500/(1, 30)/(1, 30)]","[3000/(1, 20)/(1, 20)]","[5000/(1, 20)/(1, 20)]","[10000/(50, 100)/(100, 101)]",[1/-/-]
1,7,1,56502,73459,"[1/(50, 100)/(100, 101)]","[2500/(1, 75)/(1, 75)]","[3000/(1, 50)/(1, 50)]","[3000/(1, 40)/(1, 40)]","[4500/(1, 40)/(1, 40)]","[4500/(1, 30)/(1, 30)]","[7000/(1, 30)/(1, 30)]","[7000/(1, 20)/(1, 20)]","[10000/(1, 20)/(1, 20)]","[15000/(50, 100)/(100, 101)]",[1/-/-]
2,8,1,56502,73459,"[1/(50, 100)/(100, 101)]","[2500/(1, 75)/(1, 75)]","[3000/(1, 50)/(1, 50)]","[3000/(1, 40)/(1, 40)]","[4500/(1, 40)/(1, 40)]","[4500/(1, 30)/(1, 30)]","[7000/(1, 30)/(1, 30)]","[7000/(1, 20)/(1, 20)]","[10000/(1, 20)/(1, 20)]","[15000/(50, 100)/(100, 101)]",[1/-/-]


In [99]:
# Initialize directories
data = "Test"
opt_models_dir = os.path.join(parent_dir, "Models-gp", data)
dt_models_dir = os.path.join(parent_dir, "Models-ml", "dt", data)
rf_models_dir = os.path.join(parent_dir, "Models-ml", "rf", data)

### Arc interdiction costs are set to 1

In [100]:
unit_cost = True
tag = "U"

In [101]:
# Interdiction budget
large_uc_B = [1, 10, 50, 100, 500, 1000, 2000, 2500, 3000]

In [102]:
# Results for the large networks with unit costs
cols = ["id", "k", f"B_{tag}", f"Opt_{tag}", f"Opt_T_{tag}", f"ObjVal_dt_{tag}", f"dt_T_{tag}", f"ObjVal_rf_{tag}", f"rf_T_{tag}"]
vals = []
for id, k in tqdm(zip(lt_net_df["id"], lt_net_df["k"])):
    name = f"network_{id}_{k}"
    for B in large_uc_B:
        # Set models directory
        # Optimal model
        opt_model_dir = os.path.join(opt_models_dir, f"Model_{B}{tag}")
        opt_json = load_json(name, opt_model_dir)
        opt_obj = opt_json["ObjVal"]
        opt_time = opt_json["Runtime"]
        # Decision tree model
        dt_model_dir = os.path.join(dt_models_dir, f"Model_{B}{tag}")
        dt_json = load_json(name, dt_model_dir)
        dt_obj = dt_json["ObjVal"]
        dt_time = dt_json["Runtime"]
        # Random forest model
        rf_model_dir = os.path.join(rf_models_dir, f"Model_{B}{tag}")
        rf_json = load_json(name, rf_model_dir)
        rf_obj = rf_json["ObjVal"]
        rf_time = rf_json["Runtime"]
        # Append results
        vals.append([id, k, B, opt_obj, opt_time, dt_obj, dt_time, rf_obj, rf_time])
u_large_df = pd.DataFrame(vals, columns=cols)
u_large_df = u_large_df.sort_values(by=["id", f"B_{tag}", "k"])
u_large_df[f"Opt_{tag}"] = u_large_df[f"Opt_{tag}"].replace(0, None) # Replace zero values with a NaN to avoid division by zero
u_large_df[f"Gap_dt_{tag}"] = (u_large_df[f"ObjVal_dt_{tag}"] - u_large_df[f"Opt_{tag}"]) / u_large_df[f"Opt_{tag}"] * 100
u_large_df[f"Gap_rf_{tag}"] = (u_large_df[f"ObjVal_rf_{tag}"] - u_large_df[f"Opt_{tag}"]) / u_large_df[f"Opt_{tag}"] * 100
u_large_df.round(2).to_csv(f"Results/LargeTestResults_{tag}.csv", index=False)

3it [00:04,  1.36s/it]


### Arc interdiction costs are greater that or equal to 1

In [103]:
unit_cost = False
tag = ""

In [104]:
# Interdiction budget
large_B = [5, 100, 200, 500, 1000, 2000, 3000, 4000, 5000, 7500, 10000, 15000, 20000]

In [105]:
# Results for the large networks with costs greater than or equal to 1
cols = ["id", "k", f"B_{tag}", f"Opt_{tag}", f"Opt_T_{tag}", f"ObjVal_dt_{tag}", f"dt_T_{tag}", f"ObjVal_rf_{tag}", f"rf_T_{tag}"]
vals = []
for id, k in tqdm(zip(lt_net_df["id"], lt_net_df["k"])):
    name = f"network_{id}_{k}"
    for B in large_B:
        # Set models directory
        # Optimal model
        opt_model_dir = os.path.join(opt_models_dir, f"Model_{B}{tag}")
        opt_json = load_json(name, opt_model_dir)
        opt_obj = opt_json["ObjVal"]
        opt_time = opt_json["Runtime"]
        # Decision tree model
        dt_model_dir = os.path.join(dt_models_dir, f"Model_{B}{tag}")
        dt_json = load_json(name, dt_model_dir)
        dt_obj = dt_json["ObjVal"]
        dt_time = dt_json["Runtime"]
        # Random forest model
        rf_model_dir = os.path.join(rf_models_dir, f"Model_{B}{tag}")
        rf_json = load_json(name, rf_model_dir)
        rf_obj = rf_json["ObjVal"]
        rf_time = rf_json["Runtime"]
        # Append results
        vals.append([id, k, B, opt_obj, opt_time, dt_obj, dt_time, rf_obj, rf_time])
large_df = pd.DataFrame(vals, columns=cols)
large_df = large_df.sort_values(by=["id", f"B_{tag}", "k"])
large_df[f"Opt_{tag}"] = large_df[f"Opt_{tag}"].replace(0, None) # Replace zero values with a NaN to avoid division by zero
large_df[f"Gap_dt_{tag}"] = (large_df[f"ObjVal_dt_{tag}"] - large_df[f"Opt_{tag}"]) / large_df[f"Opt_{tag}"] * 100
large_df[f"Gap_rf_{tag}"] = (large_df[f"ObjVal_rf_{tag}"] - large_df[f"Opt_{tag}"]) / large_df[f"Opt_{tag}"] * 100
large_df.round(2).to_csv(f"Results/LargeTestResults_{tag}.csv", index=False)

3it [00:07,  2.40s/it]
